Siraj 的 README.md Credits 有交代，來源是 [pannouse repo](https://github.com/pannous/tensorflow-speech-recognition) 
裡面有一堆東西，將來可以一一玩玩學習學習，其中的 `lstm-tflearn.py` 正是他給 newcomers 的入門處。
    
比起 Siraj 的 demo.py 這個要好一點點，有下列問題要解決：

- [ ] 1. while loop 也是亂寫
- [ ] 2. Train data 跟 Test data 都是同一鍋 data 難怪要發生 overfitting
- [ ] 3. training_iters = 300000 早就已經 overfitting 了 --> 如何用 tensorboard 觀察訓練狀況


放棄 Siraj 的 demo.py 改玩這個 lstm-tflearn.py

In [1]:
#!/usr/bin/env python
#!/usr/bin/env python
import tensorflow as tf

In [2]:
import tflearn

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
import speech_data

speech_data_00> 

In [4]:
import time
import peforth

epoch_count = 10
learning_rate = 0.0001
training_iters = 300000  # steps
batch_size = 64

width = 20  # mfcc features
height = 80  # (max) length of utterance
classes = 10  # digits

In [5]:
batch = word_batch = speech_data.mfcc_batch_generator(batch_size)

In [6]:

# Network building
net = tflearn.input_data([None, width, height])
net = tflearn.lstm(net, 128*4, dropout=0.8)
net = tflearn.fully_connected(net, classes, activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=learning_rate, loss='categorical_crossentropy')
model = tflearn.DNN(net, tensorboard_verbose=0)

## add this "fix" for tensorflow version errors
for x in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES): tf.add_to_collection(tf.GraphKeys.VARIABLES, x )


In [7]:
# Manually load saved model and manually predict a wave file
peforth.ok('load> ', loc=locals(),cmd='''
    :> [0] to locals 
    marker ~~~
    locals inport \ get model 
    .( restore-model \ to restore the final model 1516057900 ) cr
    : restore-model ( -- ) // Timestamp from saved_networks\tflearn.lstm.model.1516057900.meta 
        model :: load("saved_networks/tflearn.lstm.model."+"1516057900") ;
    import librosa constant librosa // ( -- module ) 
    import numpy constant np // ( -- module )  
    .( "path\\name.wav" predict . cr \ to predict the given wave file ) cr 
    : predict ( pathname -- results ) // results is an array of scores of each digit 0~9
        ( pathname ) librosa :> load(pop(),mono=True) ( y,sr )
        librosa :> feature.mfcc(tos()[0],tos()[1]) nip ( mfcc )
        np :> pad(tos(),((0,0),(0,80-len(tos()[0]))),mode='constant',constant_values=0) nip ( MFCC )
        model :> predict([pop()]) ;
    .( Run "~~~ exit" to continue ) cr \ avoid jupyter 'exit' conflict 
    ''')

restore-model \ to restore the final model 1516057900 
"path\name.wav" predict . cr \ to predict the given wave file 
Run "~~~ exit" to continue 
load> restore-model
INFO:tensorflow:Restoring parameters from c:\Users\hcche\Documents\GitHub\tensorflow_speech_recognition_demo\saved_networks\tflearn.lstm.model.1516057900
load> ~~~ exit
load> 

In [ ]:
peforth.ok(cmd='.( 'exit' command to continue but are you sure you want to continue the training?  ) cr')

In [ ]:
# Training
for i in range(1,int(training_iters/epoch_count)):
    trainX, trainY, testX, testY = next(batch)
    model.fit(
        trainX,
        trainY,
        n_epoch=epoch_count,
        validation_set=(testX, testY),
        show_metric=True,
        batch_size=batch_size)
    # must have the saved_networks/ folder
    training_step = i * epoch_count
    if training_step == 100: # I want to see it w/o a long wait
        time_stamp = int(time.time())
        model.save("saved_networks/tflearn.lstm.model."+str(time_stamp))
    if training_step % 2000 == 0:
        time_stamp = int(time.time())
        model.save("saved_networks/tflearn.lstm.model."+str(time_stamp))
        

In [11]:
_y = model.predict(next(batch)[0])  # << add your own voice here
print (_y) 

loaded batch of 2160 train_files, 240 test_files
[[ 0.07249145  0.05034124  0.39199835  0.06235743  0.08092596  0.06850597
   0.04071332  0.07827483  0.07219983  0.0821915 ]
 [ 0.27327371  0.07073745  0.0431348   0.26834419  0.04446147  0.06223009
   0.04096138  0.05492616  0.0785247   0.06340601]
 [ 0.05898944  0.0653505   0.03690695  0.14711709  0.14080052  0.258636
   0.03952236  0.12874594  0.06591355  0.05801762]
 [ 0.04906204  0.07178809  0.62906533  0.04061767  0.03376167  0.04193685
   0.0340291   0.03070006  0.04122195  0.02781727]
 [ 0.05727114  0.05281516  0.04099222  0.60353643  0.04904284  0.02926566
   0.03639036  0.03871962  0.05217351  0.03979301]
 [ 0.03953297  0.03320847  0.02793933  0.0271824   0.02082851  0.72667605
   0.0171543   0.01503931  0.0310707   0.06136799]
 [ 0.30066219  0.02614016  0.05200802  0.03553839  0.34469524  0.08536816
   0.02082197  0.02833156  0.05581613  0.05061818]
 [ 0.03971265  0.03399683  0.03227011  0.68432045  0.02518806  0.04281615
   0

In [14]:
next(batch)[0][0].shape

loaded batch of 2160 train_files, 240 test_files


(20, 80)